<h1>Import Packages</h1>

In [36]:
!pip install openai pymupdf

<h1>Upload Resume</h1>

In [37]:
from google.colab import files
import fitz  # pymupdf

print("📄 Upload your resume (PDF format)")
uploaded_resume = files.upload()
resume_path = list(uploaded_resume.keys())[0]

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

resume_text = extract_text_from_pdf(resume_path)
print("Resume text extracted successfully!")

📄 Upload your resume (PDF format)


Saving Moroney updated Resume 202503.docx.pdf to Moroney updated Resume 202503.docx (3).pdf
Resume text extracted successfully!


In [38]:
from openai import OpenAI
# Replace with own API key
api_key =
client = OpenAI(api_key=api_key)

In [39]:
import requests

def search_jobs(title="software engineer", location="seattle"):
    url = "https://jsearch.p.rapidapi.com/search"

    querystring = {
        "query": f"{title} in {location}",
        "page": "1",           # pagination available
        "num_pages": "1"
    }

    headers = {
        "x-rapidapi-key": , # Replace with own API key
        "x-rapidapi-host": "jsearch.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print("Error:", response.status_code)
        return []

In [40]:
import re

def extract_score(response_text):
    match = re.search(r"(\d{1,3})", response_text)
    return int(match.group(1)) if match else 0

def match_jobs(resume, jobs):
    results = []
    for job in jobs:
        job_desc = job.get("job_description", "")
        prompt = f"""Given the resume:
---
{resume}

Evaluate its fit for the job below and rate from 0–100. Explain briefly:
---
{job_desc}
"""
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.choices[0].message.content
        score = extract_score(content)
        results.append({**job, "score": score, "reason": content})
    return sorted(results, key=lambda x: x['score'], reverse=True)

def generate_cover_letter(resume, job_description):
    prompt = f"""Write a short, enthusiastic, tailored cover letter using the resume below for the following job.
Resume:
---
{resume}

Job Description:
---
{job_description}
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [41]:
# Customize your job search here
search_title = "software engineer"
search_location = "remote"

print(f"🔍 Searching for: {search_title} in {search_location}...")
job_listings = search_jobs(search_title, search_location)
print(f"✅ Found {len(job_listings)} jobs")
top_matches = match_jobs(resume_text, job_listings[:5])  # adjust number as needed

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"📍 Location: {job['job_city']}, {job['job_country']}")
    print(f"🔢 Match Score: {job['score']}")
    print(f"🗣 Reason: {job['reason']}")
    print("\n📝 Cover Letter:\n")
    print(generate_cover_letter(resume_text, job['job_description']))
    print("\n" + "="*80 + "\n")

🔍 Searching for: software engineer in remote...
✅ Found 10 jobs

📌 Job: Founding Software Engineer (Full-Stack, Remote) at Glass Health
📍 Location: None, None
🔢 Match Score: 85
🗣 Reason: The applicant, Chris Moroney, is a great fit for the job offered at Glass Health. Their strong background in machine learning and AI aligns well with the role, and their extensive experience in building and managing production systems addresses a key requirement mentioned in the job description. Moroney seems comfortable working in different tech environments and fulfills the need for proficiency in frontend and backend technologies mentioned in the tech stack (React.js, Python). 

However, they lack the specifically mentioned experience in TypeScript, Django, and Postgres, which could affect the rate of their initial adaptability to the job. Also, experience in a healthcare environment isn't presented and will be an area of learning. 

Based on this, I'd rate the candidate an 85 out of 100. The candid

In [42]:
# 📌 Authenticate with Google & set up APIs
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaInMemoryUpload
import datetime

creds, _ = default()
gc = gspread.authorize(creds)
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

In [43]:
# 📄 Save cover letter to Google Doc and return shareable link
def save_cover_letter_to_doc(title, content, folder_name="Generated Cover Letters"):
    # 1. Search for the folder (create it if not found)
    folder_id = None
    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed = false",
        spaces='drive',
        fields='files(id, name)',
    ).execute()
    folders = response.get('files', [])
    if folders:
        folder_id = folders[0]['id']
    else:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        folder_id = folder.get('id')

    # 2. Create the doc
    doc = docs_service.documents().create(body={"title": title}).execute()
    doc_id = doc.get("documentId")

    # 3. Move it to the folder
    drive_service.files().update(
        fileId=doc_id,
        addParents=folder_id,
        removeParents='root',
        fields='id, parents'
    ).execute()

    # 4. Write content to doc
    requests = [{"insertText": {"location": {"index": 1}, "text": content}}]
    docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

    return f"https://docs.google.com/document/d/{doc_id}/edit"


In [44]:
# 📊 Get or create Google Sheet to track job applications
def get_or_create_sheet(sheet_name="Job Tracker"):
    try:
        sheet = gc.open(sheet_name)
    except:
        sheet = gc.create(sheet_name)
        sheet.share('', perm_type='anyone', role='writer')
        sheet.sheet1.append_row(["Date", "Job Title", "Company", "Location", "Score", "Doc Link"])
    return sheet.sheet1

In [45]:
# ➕ Add a row to Google Sheet
def log_to_sheet(sheet, job, score, doc_link):
    row = [
        datetime.datetime.now().strftime("%Y-%m-%d"),
        job['job_title'],
        job['employer_name'],
        f"{job['job_city']}, {job['job_country']}",
        score,
        doc_link
    ]
    sheet.append_row(row)

In [46]:
# ✅ Run this to generate cover letters and log to Google Docs/Sheets
sheet = get_or_create_sheet("Job Tracker")

# Get and sort matched jobs ONCE
sorted_matches = sorted(top_matches, key=lambda x: x['score'], reverse=True)[:5]  # Top 5 consistent picks

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"🔢 Match Score: {job['score']}")
    print("Generating cover letter...")

    # Generate cover letter using the job description from the matched job
    cover_letter = generate_cover_letter(resume_text, job['job_description'])

    # Save to Google Doc
    doc_link = save_cover_letter_to_doc(f"{job['job_title']} - {job['employer_name']}", cover_letter)

    # Log everything to Google Sheet
    log_to_sheet(sheet, job, job['score'], doc_link)

    print(f"✅ Saved cover letter: {doc_link}")
    print("=" * 80)



📌 Job: Founding Software Engineer (Full-Stack, Remote) at Glass Health
🔢 Match Score: 85
Generating cover letter...
✅ Saved cover letter: https://docs.google.com/document/d/1WYPcjg-jyUt2d7H3yt6ZZ9nmoG9X51pCsstTI5z6BGM/edit

📌 Job: Software Engineer, Sr. (Remote | 24x7x365 Ops | DoD) at Rackner
🔢 Match Score: 65
Generating cover letter...
✅ Saved cover letter: https://docs.google.com/document/d/1de-QcRdjQ91r2pFacjDnr7WXxlPKVlee6xvLKj4Upxw/edit

📌 Job: Staff Software Engineering – Virtual Platforms (Remote/Multiple locations) at Qualcomm
🔢 Match Score: 55
Generating cover letter...
✅ Saved cover letter: https://docs.google.com/document/d/1O8jhaNpb0Cf3OiMrezPGgudW3TTBZZpWgya1LBpisjc/edit

📌 Job: Software Engineer II - Mobile iOS (REMOTE) at DICK'S Sporting Goods
🔢 Match Score: 40
Generating cover letter...
✅ Saved cover letter: https://docs.google.com/document/d/1ClsdzfdS60twHonvbT0_cMGP3gWxVxX9mV1rMspaXKw/edit

📌 Job: Software Engineer II (Remote) at The Home Depot
🔢 Match Score: 3
Gene